# Noise Sweep ROC
Monte-Carlo sweep: add shot noise, output ROC curve for TORUS side-band detection.

In [ ]:
import numpy as np, matplotlib.pyplot as plt
from scipy.signal import periodogram, hann
from scipy.spatial.distance import pdist
from sklearn.metrics import roc_curve, auc

N = 1000
L = 100
k0 = 2 * np.pi / (L / 10)
k1 = k0 * (1 + 1/14)
amp0 = 1.0
amp1 = 0.07
n_trials = 100
noise_levels = np.linspace(0, 0.5, 20)
tpr, fpr = [], []

for noise in noise_levels:
    tp = fp = fn = tn = 0
    for _ in range(n_trials):
        theta = np.random.uniform(0, 2*np.pi, N)
        r_sb = L/2 + (L/4) * (amp0 * np.sin(k0 * np.arange(N)) + amp1 * np.sin(k1 * np.arange(N)))
        x_sb = r_sb * np.cos(theta)
        y_sb = r_sb * np.sin(theta)
        dists = pdist(np.column_stack([x_sb, y_sb]))
        bins = np.linspace(0, L, 200)
        g2, _ = np.histogram(dists, bins=bins)
        g2 = g2 / g2.max()
        g2 += np.random.normal(0, noise, size=g2.shape)
        windowed = g2 * hann(len(g2))
        freqs, power = periodogram(windowed, scaling='spectrum')
        k0_idx = np.argmax(power)
        k0_val = freqs[k0_idx]
        k1_val = k0_val * (1 + 1/14)
        k1_idx = np.argmin(abs(freqs - k1_val))
        deltaP = 10 * np.log10(power[k1_idx] / power[k0_idx])
        detected = abs(k1_val-freqs[k1_idx])/k1_val<=.03 and -45<deltaP<-25
        if detected:
            tp += 1
        else:
            fn += 1
        # Null test: no side-band
        r_null = L/2 + (L/4) * (amp0 * np.sin(k0 * np.arange(N)))
        x_null = r_null * np.cos(theta)
        y_null = r_null * np.sin(theta)
        dists_null = pdist(np.column_stack([x_null, y_null]))
        g2_null, _ = np.histogram(dists_null, bins=bins)
        g2_null = g2_null / g2_null.max()
        g2_null += np.random.normal(0, noise, size=g2_null.shape)
        windowed_null = g2_null * hann(len(g2_null))
        freqs_null, power_null = periodogram(windowed_null, scaling='spectrum')
        k0_idx_null = np.argmax(power_null)
        k0_val_null = freqs_null[k0_idx_null]
        k1_val_null = k0_val_null * (1 + 1/14)
        k1_idx_null = np.argmin(abs(freqs_null - k1_val_null))
        deltaP_null = 10 * np.log10(power_null[k1_idx_null] / power_null[k0_idx_null])
        detected_null = abs(k1_val_null-freqs_null[k1_idx_null])/k1_val_null<=.03 and -45<deltaP_null<-25
        if detected_null:
    
1
        else:
    
1
    tpr.append(tp / n_trials)
    
,
,
,
,

: {
: {
: 
3
,
: 
,
: 

: {
: 
,
: 
3.1

: 4,
: 5